In [ ]:
import os
import random
import inspect
import csv
from copy import deepcopy
from dataclasses import dataclass
from typing import List, Optional, Tuple, Dict, Any

import torch
import torch.nn.functional as F
from torch import nn
import torchvision.transforms.functional as TVF
from torchvision.utils import save_image

from diffusers import DiffusionPipeline
from huggingface_hub import hf_hub_download
import open_clip

import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt


@dataclass
class RLConfig:
    model_path: str = r"C:\Users\MiguelDiaz - GenImag\Documents\proyecto IA\Generador"
    cache_dir: str = r"C:\IA RL\IA\cache"
    train_prompts_file: str = "training_prompts_expanded.txt"
    val_prompts_file: str = "validation_prompts_400.txt"
    batch_size: int = 20
    lr: float = 1e-5
    steps: int = 50
    grad_accum_steps: int = 5
    max_grad_norm: float = 1.0
    prior_steps: int = 25
    decoder_steps: int = 25
    sr_steps: int = 10
    prior_guidance: float = 4.0
    decoder_guidance: float = 8
    w_aesthetic: float = 0.4
    w_clip_align: float = 0.4
    w_pickscore: float = 0.15
    w_hpsv2: float = 0.05
    reward_norm: str = "none"
    eval_every: int = 5
    eval_dir: str = "eval_samples"
    save_n_grid: int = 4
    device: str = "cuda" if torch.cuda.is_available() else "cpu"
    k_folds: int = 4
    test_ratio: float = 0.1
    eval_batch_size: int = 2
    pickscore_model_path: Optional[str] = r"C:\Users\MiguelDiaz - GenImag\Documents\proyecto IA\Evaluadores\pickscore"
    hpsv2_model_path: Optional[str] = r"C:\Users\MiguelDiaz - GenImag\Documents\proyecto IA\Evaluadores\hpsv2\HPSv2-hf"
    seed: int = 42
    early_stopping_patience: int = 10
    min_delta: float = 0.001
    lora_rank: int = 4
    lora_alpha: int = 16
    lora_dropout: float = 0.1
    param_search_n: int = 0
    lora_rank_options: Tuple[int, ...] = (20, 4, 8, 16)
    lora_alpha_options: Tuple[int, ...] = (8, 16, 32, 64)
    lora_dropout_options: Tuple[float, ...] = (0.05, 0.1, 0.2)
    lr_options: Tuple[float, ...] = (1e-5, 5e-5, 1e-4)
    w_aesthetic_options: Tuple[float, ...] = (0.1, 0.2, 0.4, 0.6)
    w_clip_align_options: Tuple[float, ...] = (0.1, 0.2, 0.4, 0.6)
    w_pickscore_options: Tuple[float, ...] = (0.0, 0.1, 0.15, 0.07)
    w_hpsv2_options: Tuple[float, ...] = (0.0, 0.01, 0.02, 00.3)
    train_decoder_steps: Optional[int] = None
    eval_decoder_steps: Optional[int] = None

    @property
    def num_inference_steps(self) -> int:
        return self.decoder_steps


def load_unclip_pipeline(cfg: RLConfig) -> DiffusionPipeline:
    is_windows_path = (
        cfg.model_path.startswith(('C:', 'D:', 'E:', 'F:', '\\', '/')) or
        ':\\' in cfg.model_path or
        cfg.model_path.startswith('.')
    )
    if is_windows_path:
        model_path_normalized = os.path.normpath(cfg.model_path)
        model_index_exists = os.path.exists(os.path.join(model_path_normalized, "model_index.json"))
        if model_index_exists:
            print(f"✅ Cargando desde ruta local: {model_path_normalized}")
            pipe = DiffusionPipeline.from_pretrained(
                model_path_normalized,
                torch_dtype=torch.float16,
                trust_remote_code=True,
            ).to(cfg.device)
        else:
            raise FileNotFoundError(
                f"❌ La ruta local {model_path_normalized} no contiene 'model_index.json'.\n"
                "Para descargar Karlo primero, ejecuta:\n"
                "  from diffusers import DiffusionPipeline\n"
                "  pipe = DiffusionPipeline.from_pretrained('kakaobrain/karlo-v1-alpha')\n"
                f"  pipe.save_pretrained(r'{model_path_normalized}')"
            )
    else:
        print(f"📥 Descargando modelo desde HuggingFace Hub: {cfg.model_path}")
        pipe = DiffusionPipeline.from_pretrained(
            cfg.model_path,
            torch_dtype=torch.float16,
            cache_dir=cfg.cache_dir,
            trust_remote_code=True,
        ).to(cfg.device)
    pipe._offload_device = None
    pipe.enable_model_cpu_offload = lambda *args, **kwargs: None
    print(f"✅ Pipeline cargada: {type(pipe).__name__}")
    return pipe


def get_decoder_unet(pipe: DiffusionPipeline):
    if hasattr(pipe, "decoder"):
        dec = pipe.decoder
        return dec.unet if hasattr(dec, "unet") else dec
    if hasattr(pipe, "unet"):
        return pipe.unet
    raise RuntimeError("No se encontró un UNet de decoder en la pipeline.")


def set_decoder_unet(pipe: DiffusionPipeline, new_unet: nn.Module) -> None:
    if hasattr(pipe, "decoder"):
        if hasattr(pipe.decoder, "unet"):
            pipe.decoder.unet = new_unet
        else:
            pipe.decoder = new_unet
    elif hasattr(pipe, "unet"):
        pipe.unet = new_unet
    else:
        raise RuntimeError("No se pudo asignar el UNet de decoder en la pipeline.")


def add_lora_to_decoder(
    pipe: DiffusionPipeline,
    r: Optional[int] = None,
    alpha: Optional[int] = None,
    dropout: Optional[float] = None,
    cfg: Optional[RLConfig] = None,
) -> None:
    if cfg is not None:
        r = getattr(cfg, 'lora_rank', 8) if r is None else r
        alpha = getattr(cfg, 'lora_alpha', 16) if alpha is None else alpha
        dropout = getattr(cfg, 'lora_dropout', 0.1) if dropout is None else dropout
    else:
        r = 8 if r is None else r
        alpha = 16 if alpha is None else alpha
        dropout = 0.1 if dropout is None else dropout
    print(f"🔧 LoRA Configuration: r={r}, alpha={alpha}, dropout={dropout}")

    try:
        from peft import LoraConfig, get_peft_model
    except ImportError:
        raise ImportError("Instala 'peft': pip install peft")

    dec_unet = get_decoder_unet(pipe)

    linear_names = [name for name, m in dec_unet.named_modules() if isinstance(m, nn.Linear)]
    preferred = [
        "to_q", "to_k", "to_v",
        "to_out.0",
        "q_proj", "k_proj", "v_proj", "out_proj",
    ]
    targets = sorted({name for name in linear_names if any(p in name for p in preferred)})
    if not targets:
        targets = linear_names
        print(f"[Aviso] No se hallaron patrones típicos; aplicaré LoRA a TODOS los Linear del decoder ({len(targets)}).")
    else:
        print(f"[LoRA] Módulos Linear objetivo detectados ({len(targets)}):")
        for t in targets[:20]:
            print("   -", t)
        if len(targets) > 20:
            print(f"   ... y {len(targets)-20} más")

    lora_cfg = LoraConfig(
        r=r,
        lora_alpha=alpha,
        target_modules=targets,
        lora_dropout=dropout,
        bias="none",
    )

    dec_unet_lora = get_peft_model(dec_unet, lora_cfg)
    set_decoder_unet(pipe, dec_unet_lora)

    for attr in ["text_encoder", "prior", "super_res_first", "super_res_last"]:
        m = getattr(pipe, attr, None)
        if m is None:
            continue
        if hasattr(m, "unet"):
            for p in m.unet.parameters():
                p.requires_grad = False
        if hasattr(m, "model"):
            for p in m.model.parameters():
                p.requires_grad = False
        if hasattr(m, "parameters"):
            for p in m.parameters():
                p.requires_grad = False

    for vae_attr in ["vae", "decoder_vae"]:
        m = getattr(pipe, vae_attr, None)
        if m is not None and hasattr(m, "parameters"):
            for p in m.parameters():
                p.requires_grad = False

    dec_unet_lora = get_decoder_unet(pipe)
    trainable_params = 0
    frozen_params = 0
    for name, p in dec_unet_lora.named_parameters():
        if "lora_" in name:
            p.requires_grad = True
            trainable_params += p.numel()
        else:
            p.requires_grad = False
            frozen_params += p.numel()
    print(f"LoRA aplicada — parámetros entrenables en UNet (solo LoRA): {trainable_params:,}")
    print(f"Parámetros congelados en UNet (base): {frozen_params:,}")


def load_clip_and_aesthetic_predictor(device: str):
    path = hf_hub_download(
        repo_id="trl-lib/ddpo-aesthetic-predictor",
        filename="aesthetic-model.pth",
        repo_type="model",
    )
    raw = torch.load(path, map_location="cpu")
    if isinstance(raw, dict) and "state_dict" in raw and isinstance(raw["state_dict"], dict):
        sd = raw["state_dict"]
    elif isinstance(raw, dict):
        sd = raw
    else:
        raise ValueError(f"Formato desconocido del checkpoint estético: tipo={type(raw)}")
    sd = {k.replace("module.", "").replace("model.", ""): v for k, v in sd.items()}
    candidates = [
        ("linear.weight", "linear.bias"),
        ("lin.weight", "lin.bias"),
        ("aesthetic_head.weight", "aesthetic_head.bias"),
        ("weight", "bias"),
    ]
    w = b = None
    for kw, kb in candidates:
        if kw in sd and kb in sd:
            w, b = sd[kw], sd[kb]
            break
    if w is None or b is None:
        w_keys = [k for k in sd.keys() if k.endswith("weight")]
        b_keys = [k for k in sd.keys() if k.endswith("bias")]
        if w_keys and b_keys:
            w, b = sd[w_keys[0]], sd[b_keys[0]]
        else:
            raise KeyError("No se encontraron claves de weight/bias en el predictor estético.")
    w = w.float(); b = b.float()
    if w.dim() == 1:
        w = w.view(1, -1)
    elif w.dim() == 2 and w.shape[0] > 1:
        w = w.mean(dim=0, keepdim=True)
    elif w.dim() == 2 and w.shape[0] == 1:
        pass
    else:
        raise ValueError(f"Forma inesperada de weight: {tuple(w.shape)}")
    if b.numel() != 1:
        b = b.flatten().mean().view(1)
    else:
        b = b.view(1)
    clip_model, _, _ = open_clip.create_model_and_transforms(
        "ViT-L-14", pretrained="openai", cache_dir=None
    )
    clip_model = clip_model.to(device)
    clip_model.eval()
    tokenizer = open_clip.get_tokenizer("ViT-L-14")
    try:
        emb_dim = clip_model.visual.output_dim
    except Exception:
        emb_dim = 768
    if w.shape[1] != emb_dim:
        print(f"[Aviso] Dimensión de weight {w.shape[1]} != emb_dim {emb_dim}. Ajustando.")
        if w.shape[1] > emb_dim:
            w = w[:, :emb_dim].contiguous()
        else:
            pad = torch.zeros(1, emb_dim - w.shape[1], device=w.device, dtype=w.dtype)
            w = torch.cat([w, pad], dim=1)
    w = w.to(device); b = b.to(device)
    return clip_model, w, b, tokenizer


def load_pickscore_model(device: str):
    class _DummyPickScore:
        def to(self, *_, **__): return self
        def score(self, prompt: str, image): return 0.0
    try:
        from imscore import PickScore
        model = PickScore()
        try:
            model = model.to(device)
        except Exception:
            pass
        return model
    except Exception:
        pass
    try:
        import pickscore
        if hasattr(pickscore, "load_model"):
            model = pickscore.load_model(device=device)
            return model
    except Exception:
        pass
    print("[Aviso] PickScore no está instalado; se ignorará w_pickscore.")
    return _DummyPickScore()


def load_hps_v2_model(device: str):
    class _DummyHPS:
        def to(self, *_, **__): return self
        def score(self, prompt: str, image): return 0.0
    try:
        from imscore import HPS
        model = HPS()
        try:
            model = model.to(device)
        except Exception:
            pass
        return model
    except Exception:
        pass
    try:
        from imscore import HPSv2
        model = HPSv2()
        try:
            model = model.to(device)
        except Exception:
            pass
        return model
    except Exception:
        pass
    print("[Aviso] HPS v2 no está instalado; se ignorará w_hpsv2.")
    return _DummyHPS()


def load_pickscore_model_real(device: str, model_path: str):
    try:
        from transformers import AutoProcessor, AutoModel
        processor = AutoProcessor.from_pretrained(model_path)
        model = AutoModel.from_pretrained(model_path).eval().to(device)
        class _PickScoreLocal:
            def __init__(self, processor, model):
                self.processor = processor; self.model = model
            def to(self, *_args, **_kwargs): return self
            def score(self, prompt: str, image):
                image_inputs = self.processor(images=[image], return_tensors="pt").to(device)
                text_inputs = self.processor(text=[prompt], return_tensors="pt").to(device)
                with torch.no_grad():
                    image_embs = self.model.get_image_features(**image_inputs)
                    image_embs = image_embs / image_embs.norm(dim=-1, keepdim=True)
                    text_embs = self.model.get_text_features(**text_inputs)
                    text_embs = text_embs / text_embs.norm(dim=-1, keepdim=True)
                    scores = self.model.logit_scale.exp() * (text_embs @ image_embs.T)[0]
                return float(scores.item())
        print(f"✅ PickScore cargado correctamente desde '{model_path}'")
        return _PickScoreLocal(processor, model)
    except Exception as e:
        print(f"❌ No se pudo cargar PickScore desde la ruta '{model_path}': {e}")
        class _Dummy:
            def to(self, *_args, **_kwargs): return self
            def score(self, prompt, image): return 0.0
        return _Dummy()


def load_hpsv2_model_real(device: str, model_path: str):
    try:
        from transformers import AutoProcessor, AutoModel
        processor = AutoProcessor.from_pretrained(model_path)
        model = AutoModel.from_pretrained(model_path).eval().to(device)
        class _HPSv2Local:
            def __init__(self, processor, model):
                self.processor = processor; self.model = model
            def to(self, *_args, **_kwargs): return self
            def score(self, prompt: str, image):
                image_inputs = self.processor(images=[image], return_tensors="pt").to(device)
                text_inputs = self.processor(text=[prompt], return_tensors="pt").to(device)
                with torch.no_grad():
                    image_embs = self.model.get_image_features(**image_inputs)
                    image_embs = image_embs / image_embs.norm(dim=-1, keepdim=True)
                    text_embs = self.model.get_text_features(**text_inputs)
                    text_embs = text_embs / text_embs.norm(dim=-1, keepdim=True)
                    scores = self.model.logit_scale.exp() * (text_embs @ image_embs.T)[0]
                return float(scores.item())
        print(f"✅ HPSv2 cargado correctamente desde '{model_path}'")
        return _HPSv2Local(processor, model)
    except Exception as e:
        print(f"❌ No se pudo cargar HPSv2 desde la ruta '{model_path}': {e}")
        class _Dummy:
            def to(self, *_args, **_kwargs): return self
            def score(self, prompt, image): return 0.0
        return _Dummy()


@dataclass
class Evaluators:
    clip_model: Any
    weight: torch.Tensor
    bias: torch.Tensor
    tokenizer: Any
    pick_model: Any
    hps_model: Any


def load_all_evaluators(cfg: RLConfig) -> Evaluators:
    print("📥 Cargando evaluadores (una sola vez)...")
    clip_model, weight, bias, tokenizer = load_clip_and_aesthetic_predictor(cfg.device)

    if getattr(cfg, "pickscore_model_path", None):
        pick_model = load_pickscore_model_real(cfg.device, cfg.pickscore_model_path)
    else:
        pick_model = load_pickscore_model(cfg.device)

    if getattr(cfg, "hpsv2_model_path", None):
        hps_model = load_hpsv2_model_real(cfg.device, cfg.hpsv2_model_path)
    else:
        hps_model = load_hps_v2_model(cfg.device)

    print("✅ Evaluadores listos")
    return Evaluators(
        clip_model=clip_model,
        weight=weight,
        bias=bias,
        tokenizer=tokenizer,
        pick_model=pick_model,
        hps_model=hps_model,
    )


def _to_01(imgs: torch.Tensor) -> torch.Tensor:
    if imgs.min() < 0:
        imgs = (imgs.clamp(-1, 1) + 1) / 2
    return imgs.clamp(0, 1)


def aesthetic_score_from_tensor(
    clip_model,
    weight: torch.Tensor,
    bias: torch.Tensor,
    images_pt: torch.Tensor,
) -> torch.Tensor:
    imgs = _to_01(images_pt).float()
    imgs = F.interpolate(imgs, size=224, mode="bicubic", align_corners=False)
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073], device=imgs.device)[None, :, None, None]
    std = torch.tensor([0.26862954, 0.26130258, 0.27577711], device=imgs.device)[None, :, None, None]
    imgs = (imgs - mean) / std
    emb = clip_model.encode_image(imgs)
    emb = emb / emb.norm(dim=-1, keepdim=True)
    scores = (emb @ weight.t()).squeeze(-1) + bias.squeeze()
    return scores.float()


def clip_text_image_alignment(
    clip_model,
    tokenizer,
    images_pt: torch.Tensor,
    prompts: List[str],
) -> torch.Tensor:
    imgs = _to_01(images_pt).float()
    imgs = F.interpolate(imgs, size=224, mode="bicubic", align_corners=False)
    mean = torch.tensor([0.48145466, 0.4578275, 0.40821073], device=imgs.device)[None, :, None, None]
    std = torch.tensor([0.26862954, 0.26130258, 0.27577711], device=imgs.device)[None, :, None, None]
    imgs = (imgs - mean) / std
    img_feats = clip_model.encode_image(imgs)
    img_feats = img_feats / img_feats.norm(dim=-1, keepdim=True)
    with torch.no_grad():
        toks = tokenizer(prompts)
        if not torch.is_tensor(toks):
            toks = torch.tensor(toks)
        toks = toks.to(imgs.device)
        if toks.dim() == 1:
            toks = toks.unsqueeze(0)
        txt_feats = clip_model.encode_text(toks)
        txt_feats = txt_feats / txt_feats.norm(dim=-1, keepdim=True)
    sims = (img_feats * txt_feats).sum(dim=-1)
    return sims.float()


def normalize_metric_per_batch(values: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
    if values.numel() <= 1:
        return torch.tanh(values)

    mean = values.mean()
    std = values.std().clamp(min=eps)
    z = (values - mean) / std
    z = torch.clamp(z, -5, 5)
    normalized = torch.tanh(z)

    if torch.isnan(normalized).any():
        print(f"⚠️ NaN en normalize_metric_per_batch: {values}")
        return torch.zeros_like(values)

    return normalized


def compute_reward_combined(
    prompts: List[str],
    images_m1_1: torch.Tensor,
    clip_model,
    tokenizer,
    weight: torch.Tensor,
    bias: torch.Tensor,
    pick_model,
    hps_model,
    w_aesthetic: float,
    w_clip_align: float,
    w_pickscore: float,
    w_hpsv2: float,
) -> torch.Tensor:
    aest = aesthetic_score_from_tensor(clip_model, weight, bias, images_m1_1)
    align = clip_text_image_alignment(clip_model, tokenizer, images_m1_1, prompts)

    pick_vals = []
    hps_vals = []
    imgs_01 = _to_01(images_m1_1)

    for p, img in zip(prompts, imgs_01.cpu()):
        pil = TVF.to_pil_image(img)

        with torch.no_grad():
            try:
                pv = float(pick_model.score(p, pil))
                pick_vals.append(pv)
            except Exception:
                pick_vals.append(0.5)

            try:
                hv = float(hps_model.score(p, pil))
                hps_vals.append(hv)
            except Exception:
                hps_vals.append(0.5)

    pick_t = torch.tensor(pick_vals, device=images_m1_1.device, dtype=aest.dtype)
    hps_t = torch.tensor(hps_vals, device=images_m1_1.device, dtype=aest.dtype)

    aest_norm = normalize_metric_per_batch(aest)
    align_norm = normalize_metric_per_batch(align)
    pick_norm = normalize_metric_per_batch(pick_t)
    hps_norm = normalize_metric_per_batch(hps_t)

    combined = (
        w_aesthetic * aest_norm +
        w_clip_align * align_norm +
        w_pickscore * pick_norm +
        w_hpsv2 * hps_norm
    )

    if torch.isnan(combined).any():
        print(f"⚠️ NaN en combined reward")
        print(f"  aest_norm: {aest_norm}")
        print(f"  align_norm: {align_norm}")
        print(f"  pick_norm: {pick_norm}")
        print(f"  hps_norm: {hps_norm}")
        return torch.zeros_like(combined)

    return combined


def normalize_rewards(
    rewards: torch.Tensor,
    mode: str = "batch",
    eps: float = 1e-6,
) -> torch.Tensor:
    if mode == "none":
        return rewards
    if mode == "tanh":
        return torch.tanh(rewards)
    mean = rewards.mean()
    std = rewards.std().clamp_min(eps)
    if std < 1e-5:
        print(f"[Aviso] Recompensas muy similares (std={std:.2e}). Retornando como están.")
        return rewards
    normalized = (rewards - mean) / std
    return normalized.clamp(-10, 10)


def load_prompts(filepath: str) -> List[str]:
    prompts: List[str] = []
    try:
        with open(filepath, "r", encoding="utf-8") as f:
            prompts = [line.strip() for line in f.readlines() if line.strip()]
        print(f"✅ Cargados {len(prompts)} prompts desde: {filepath}")
    except FileNotFoundError:
        print(f"❌ Error: No se encontró el archivo {filepath}")
    except Exception as e:
        print(f"❌ Error al leer {filepath}: {type(e).__name__}: {e}")
    return prompts


def prepare_cv_datasets(cfg: RLConfig) -> Tuple[List[str], List[str]]:
    train_prompts = load_prompts(cfg.train_prompts_file)
    val_prompts = load_prompts(cfg.val_prompts_file)
    all_prompts = train_prompts + val_prompts
    if not all_prompts:
        print("❌ Error: No se cargaron prompts de entrenamiento/validación.")
        return [], []
    random.seed(cfg.seed)
    random.shuffle(all_prompts)
    test_size = max(1, int(len(all_prompts) * cfg.test_ratio)) if cfg.test_ratio > 0 else 0
    test_prompts = all_prompts[:test_size]
    cv_prompts = all_prompts[test_size:]
    print(f"🚧 División de datos: total={len(all_prompts)}, test={len(test_prompts)}, cv={len(cv_prompts)}")
    return test_prompts, cv_prompts


def kfold_split(prompts: List[str], k: int, seed: int = 42) -> List[Tuple[List[str], List[str]]]:
    n = len(prompts)
    if k <= 1 or n < k:
        return [(prompts, [])]
    shuffled_prompts = prompts.copy()
    random.Random(seed).shuffle(shuffled_prompts)
    fold_size = n // k
    splits: List[Tuple[List[str], List[str]]] = []
    for i in range(k):
        val_start = i * fold_size
        val_end = (i + 1) * fold_size if i < k - 1 else n
        val_prompts = shuffled_prompts[val_start:val_end]
        train_prompts = shuffled_prompts[:val_start] + shuffled_prompts[val_end:]
        splits.append((train_prompts, val_prompts))
    return splits


def call_pipe_compat(
    pipe: DiffusionPipeline,
    prompts: List[str],
    cfg: RLConfig,
    want_pt: bool = True,
    override_prior_steps: Optional[int] = None,
    override_decoder_steps: Optional[int] = None,
    override_sr_steps: Optional[int] = None,
) -> torch.Tensor:
    sig = inspect.signature(pipe.__call__).parameters
    kwargs: Dict[str, Any] = {}
    if "prompt" in sig:
        kwargs["prompt"] = prompts
    elif "text" in sig:
        kwargs["text"] = prompts
    prior_steps = max(2, int(override_prior_steps if override_prior_steps is not None else cfg.prior_steps))
    decoder_steps = max(2, int(override_decoder_steps if override_decoder_steps is not None else cfg.decoder_steps))
    sr_steps = max(2, int(override_sr_steps if override_sr_steps is not None else cfg.sr_steps))
    if "prior_num_inference_steps" in sig:
        kwargs["prior_num_inference_steps"] = prior_steps
    if "decoder_num_inference_steps" in sig:
        kwargs["decoder_num_inference_steps"] = decoder_steps
    if "super_res_num_inference_steps" in sig:
        kwargs["super_res_num_inference_steps"] = sr_steps
    if "prior_guidance_scale" in sig:
        kwargs["prior_guidance_scale"] = getattr(cfg, "prior_guidance", 4.0)
    if "decoder_guidance_scale" in sig:
        kwargs["decoder_guidance_scale"] = getattr(cfg, "decoder_guidance", 7.5)
    if want_pt and "output_type" in sig:
        kwargs["output_type"] = "pt"
        kwargs["return_dict"] = True
    print(
        f"[UnCLIP] steps -> prior={prior_steps}, decoder={decoder_steps}, super_res={sr_steps}"
    )
    out = pipe(**kwargs)
    images = getattr(out, "images", out)
    if isinstance(images, list):
        images = torch.stack([TVF.to_tensor(img) for img in images], dim=0)
    elif isinstance(images, torch.Tensor):
        pass
    else:
        images = torch.from_numpy(images).permute(0, 3, 1, 2).contiguous()
    return images.to(next(get_decoder_unet(pipe).parameters()).device)


def evaluate_prompts(
    pipe: DiffusionPipeline,
    prompts: List[str],
    cfg: RLConfig,
    clip_model,
    tokenizer,
    weight: torch.Tensor,
    bias: torch.Tensor,
    pick_model,
    hps_model,
) -> Dict[str, float]:
    if not prompts:
        return {
            "aesthetic": 0.0,
            "clip_align": 0.0,
            "pickscore": 0.0,
            "hpsv2": 0.0,
            "aesthetic_norm": 0.0,
            "clip_align_norm": 0.0,
            "pickscore_norm": 0.0,
            "hpsv2_norm": 0.0,
            "weighted": 0.0,
        }
    get_decoder_unet(pipe).eval()
    total_aest = 0.0; total_align = 0.0; total_pick = 0.0; total_hps = 0.0
    total_aest_norm = 0.0; total_align_norm = 0.0; total_pick_norm = 0.0; total_hps_norm = 0.0
    count = 0
    eval_batch_size = max(1, getattr(cfg, 'eval_batch_size', 2))
    for i in range(0, len(prompts), eval_batch_size):
        batch_prompts = prompts[i:i + eval_batch_size]
        with torch.no_grad():
            imgs01_batch = call_pipe_compat(
                pipe,
                batch_prompts,
                cfg,
                want_pt=True,
                override_decoder_steps=(
                    cfg.eval_decoder_steps if cfg.eval_decoder_steps is not None else cfg.decoder_steps
                ),
            )
        imgs_m1_1_batch = imgs01_batch * 2 - 1
        with torch.no_grad():
            aest_batch = aesthetic_score_from_tensor(clip_model, weight, bias, imgs_m1_1_batch).detach().cpu()
            align_batch = clip_text_image_alignment(clip_model, tokenizer, imgs_m1_1_batch, batch_prompts).detach().cpu()
        total_aest += float(aest_batch.sum())
        total_align += float(align_batch.sum())
        total_aest_norm += float(torch.tanh(aest_batch).sum())
        total_align_norm += float(torch.tanh(align_batch).sum())
        pick_vals: List[float] = []
        hps_vals: List[float] = []

        for p, img_t in zip(batch_prompts, imgs01_batch.detach().cpu()):
            pil = TVF.to_pil_image(img_t)
            with torch.no_grad():
                try:
                    pv = float(pick_model.score(p, pil))
                    pick_vals.append(pv)
                except Exception:
                    pick_vals.append(0.0)
                try:
                    hv = float(hps_model.score(p, pil))
                    hps_vals.append(hv)
                except Exception:
                    hps_vals.append(0.0)

        if pick_vals:
            pick_tensor = torch.tensor(pick_vals, device=aest_batch.device, dtype=aest_batch.dtype)
            pick_norm_tensor = normalize_metric_per_batch(pick_tensor)
            total_pick += float(pick_tensor.sum())
            total_pick_norm += float(pick_norm_tensor.sum())

        if hps_vals:
            hps_tensor = torch.tensor(hps_vals, device=aest_batch.device, dtype=aest_batch.dtype)
            hps_norm_tensor = normalize_metric_per_batch(hps_tensor)
            total_hps += float(hps_tensor.sum())
            total_hps_norm += float(hps_norm_tensor.sum())

        count += len(batch_prompts)
        del imgs01_batch, imgs_m1_1_batch, aest_batch, align_batch
        torch.cuda.empty_cache()
    if count > 0:
        aest_mean = total_aest / count
        align_mean = total_align / count
        pick_mean = total_pick / count
        hps_mean = total_hps / count
        aest_norm_mean = total_aest_norm / count
        align_norm_mean = total_align_norm / count
        pick_norm_mean = total_pick_norm / count
        hps_norm_mean = total_hps_norm / count
    else:
        aest_mean = align_mean = pick_mean = hps_mean = 0.0
        aest_norm_mean = align_norm_mean = pick_norm_mean = hps_norm_mean = 0.0
    weighted = (
        cfg.w_aesthetic * aest_norm_mean +
        cfg.w_clip_align * align_norm_mean +
        cfg.w_pickscore * pick_norm_mean +
        cfg.w_hpsv2 * hps_norm_mean
    )
    return {
        "aesthetic": aest_mean,
        "clip_align": align_mean,
        "pickscore": pick_mean,
        "hpsv2": hps_mean,
        "aesthetic_norm": aest_norm_mean,
        "clip_align_norm": align_norm_mean,
        "pickscore_norm": pick_norm_mean,
        "hpsv2_norm": hps_norm_mean,
        "weighted": weighted,
    }


def rl_training_loop_for_prompts(
    cfg: RLConfig,
    train_prompts: List[str],
    val_prompts: List[str],
    fold_index: int,
    results_dir: str = "cv_results",
    evaluators: Optional[Evaluators] = None,
) -> Dict[str, Any]:
    device = cfg.device
    os.makedirs(results_dir, exist_ok=True)
    fold_dir = os.path.join(results_dir, f"fold_{fold_index:02d}")
    os.makedirs(fold_dir, exist_ok=True)

    pipe = load_unclip_pipeline(cfg)
    add_lora_to_decoder(pipe, cfg=cfg)
    unet = get_decoder_unet(pipe)
    unet.train()
    trainable_params = [p for p in unet.parameters() if p.requires_grad]
    if not trainable_params:
        raise RuntimeError("❌ No hay parámetros entrenables en el decoder")
    optimizer = torch.optim.AdamW(trainable_params, lr=cfg.lr)

    if evaluators is None:
        evaluators = load_all_evaluators(cfg)

    clip_model = evaluators.clip_model
    weight = evaluators.weight
    bias = evaluators.bias
    tokenizer = evaluators.tokenizer
    pick_model = evaluators.pick_model
    hps_model = evaluators.hps_model

    print(f"🚀 Iniciando entrenamiento del fold {fold_index+1}/{cfg.k_folds}...")

    train_metrics_history: List[Dict[str, float]] = []
    val_metrics_history: List[Dict[str, float]] = []
    best_weighted_score = -float('inf')
    patience_counter = 0
    best_model_state = None

    for step in range(1, cfg.steps + 1):
        batch = random.sample(train_prompts, k=min(cfg.batch_size, len(train_prompts)))
        optimizer.zero_grad(set_to_none=True)
        micro_losses: List[float] = []
        raw_rewards_all: List[float] = []
        for _ in range(cfg.grad_accum_steps):
            dec_steps_train = (
                cfg.train_decoder_steps
                if cfg.train_decoder_steps is not None
                else cfg.decoder_steps
            )
            torch.cuda.empty_cache()
            imgs01 = call_pipe_compat(
                pipe,
                batch,
                cfg,
                want_pt=True,
                override_decoder_steps=dec_steps_train,
            )
            imgs_m1_1 = imgs01 * 2 - 1
            rewards = compute_reward_combined(
                prompts=batch,
                images_m1_1=imgs_m1_1,
                clip_model=clip_model,
                tokenizer=tokenizer,
                weight=weight,
                bias=bias,
                pick_model=pick_model,
                hps_model=hps_model,
                w_aesthetic=cfg.w_aesthetic,
                w_clip_align=cfg.w_clip_align,
                w_pickscore=cfg.w_pickscore,
                w_hpsv2=cfg.w_hpsv2,
            )
            with torch.no_grad():
                raw_mean = float(rewards.detach().cpu().mean().item())
            raw_rewards_all.append(raw_mean)
            rewards_norm = normalize_rewards(rewards, cfg.reward_norm)
            loss = -rewards_norm.mean() / cfg.grad_accum_steps
            try:
                loss.backward()
            except Exception as e:
                print(f"[Aviso] Backprop falló: {type(e).__name__}: {e}")
                micro_losses.append(float(loss.detach().cpu().item()))
                continue
            micro_losses.append(float(loss.detach().cpu().item()))
            del imgs01, imgs_m1_1, rewards, rewards_norm, loss
            torch.cuda.empty_cache()
        try:
            torch.nn.utils.clip_grad_norm_(trainable_params, cfg.max_grad_norm)
            optimizer.step()
        except Exception as e:
            print(f"[Aviso] Optimizer step falló: {type(e).__name__}: {e}")
        try:
            train_metrics_step = evaluate_prompts(
                pipe,
                batch,
                cfg,
                clip_model,
                tokenizer,
                weight,
                bias,
                pick_model,
                hps_model,
            )
        except Exception:
            train_metrics_step = {
                "weighted": 0.0,
                "aesthetic": 0.0,
                "clip_align": 0.0,
                "pickscore": 0.0,
                "hpsv2": 0.0,
                "aesthetic_norm": 0.0,
                "clip_align_norm": 0.0,
                "pickscore_norm": 0.0,
                "hpsv2_norm": 0.0,
            }
        train_metrics_history.append(train_metrics_step)
        try:
            val_metrics_step = evaluate_prompts(
                pipe,
                val_prompts,
                cfg,
                clip_model,
                tokenizer,
                weight,
                bias,
                pick_model,
                hps_model,
            )
        except Exception:
            val_metrics_step = {
                "weighted": 0.0,
                "aesthetic": 0.0,
                "clip_align": 0.0,
                "pickscore": 0.0,
                "hpsv2": 0.0,
                "aesthetic_norm": 0.0,
                "clip_align_norm": 0.0,
                "pickscore_norm": 0.0,
                "hpsv2_norm": 0.0,
            }
        val_metrics_history.append(val_metrics_step)
        current_weighted = val_metrics_step['weighted']
        if current_weighted > best_weighted_score + cfg.min_delta:
            best_weighted_score = current_weighted
            patience_counter = 0
            best_model_state = get_decoder_unet(pipe).state_dict().copy()
            print(f"✅ Mejora detectada! Nuevo mejor score: {best_weighted_score:.4f}")
        else:
            patience_counter += 1
            print(f"⏳ Sin mejora ({patience_counter}/{cfg.early_stopping_patience}) - Mejor score: {best_weighted_score:.4f}")
        if patience_counter >= cfg.early_stopping_patience:
            print(f"🛑 Early stopping activado después de {step} pasos sin mejora")
            if best_model_state is not None:
                get_decoder_unet(pipe).load_state_dict(best_model_state)
            break
        avg_loss = sum(micro_losses) / len(micro_losses) if micro_losses else float('nan')
        avg_raw_reward = sum(raw_rewards_all) / len(raw_rewards_all) if raw_rewards_all else 0.0
        print(
            f"[Fold {fold_index+1}, Paso {step:03d}] "
            f"Loss: {avg_loss:.4f}, Reward: {avg_raw_reward:.4f} | "
            f"Train -> aest: {train_metrics_step['aesthetic']:.4f}, align: {train_metrics_step['clip_align']:.4f}, "
            f"pick: {train_metrics_step['pickscore']:.4f}, hps: {train_metrics_step['hpsv2']:.4f}, "
            f"weighted: {train_metrics_step['weighted']:.4f} | "
            f"Val -> aest: {val_metrics_step['aesthetic']:.4f}, align: {val_metrics_step['clip_align']:.4f}, "
            f"pick: {val_metrics_step['pickscore']:.4f}, hps: {val_metrics_step['hpsv2']:.4f}, "
            f"weighted: {val_metrics_step['weighted']:.4f}"
        )
        torch.cuda.empty_cache()
    if best_model_state is not None:
        get_decoder_unet(pipe).load_state_dict(best_model_state)
    lora_path = os.path.join(fold_dir, f"lora_weights_fold_{fold_index:02d}.pth")
    os.makedirs(os.path.dirname(lora_path), exist_ok=True)

    unet = get_decoder_unet(pipe)
    lora_weights = {k: v for k, v in unet.state_dict().items() if "lora_" in k}
    payload = {
        "lora_state_dict": lora_weights,
        "lora_rank": cfg.lora_rank,
        "lora_alpha": cfg.lora_alpha,
        "lora_dropout": cfg.lora_dropout,
    }
    torch.save(payload, lora_path)
    print(f"✅ Guardados {len(lora_weights)} pesos LoRA en {lora_path}")
    print(f"✅ Fold {fold_index+1}: entrenamiento completado. Pesos guardados en {lora_path}")
    val_metrics = evaluate_prompts(
        pipe,
        val_prompts,
        cfg,
        clip_model,
        tokenizer,
        weight,
        bias,
        pick_model,
        hps_model,
    )
    return {
        "fold": fold_index,
        "metrics": val_metrics,
        "weights_path": lora_path,
        "train_metrics_history": train_metrics_history,
        "val_metrics_history": val_metrics_history,
        "best_weighted_score": best_weighted_score,
        "early_stopped": patience_counter >= cfg.early_stopping_patience,
    }


def cross_validation_training(
    cfg: RLConfig,
    k_folds: Optional[int] = None,
    test_ratio: Optional[float] = None,
    results_dir: str = "cv_results",
    return_results: bool = True,
    evaluators: Optional[Evaluators] = None,
) -> Dict[str, Any]:
    if k_folds is None:
        k_folds = cfg.k_folds
    if test_ratio is None:
        test_ratio = cfg.test_ratio
    cfg.k_folds = k_folds
    cfg.test_ratio = test_ratio

    random.seed(cfg.seed)
    torch.manual_seed(cfg.seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(cfg.seed)

    print(f"\n🎲 Realizando validación cruzada con {k_folds} folds y test_ratio={test_ratio:.2f}\n")
    print(f"⚡ Early stopping configurado: paciencia={cfg.early_stopping_patience}, min_delta={cfg.min_delta}\n")

    train_prompts_file_prompts = load_prompts(cfg.train_prompts_file)
    val_prompts_file_prompts = load_prompts(cfg.val_prompts_file)
    all_prompts = train_prompts_file_prompts + val_prompts_file_prompts
    if not all_prompts:
        print("❌ Error: No se cargaron prompts de entrenamiento/validación.")
        return {}

    random.shuffle(all_prompts)
    test_prompts: List[str] = []
    if test_ratio and test_ratio > 0:
        test_size = max(1, int(len(all_prompts) * test_ratio))
        test_prompts = all_prompts[:test_size]
        cv_prompts = all_prompts[test_size:]
    else:
        cv_prompts = all_prompts

    if not cv_prompts:
        print("❌ Abortando cross-validation: no hay datos suficientes para dividir en folds.")
        return {}

    splits = kfold_split(cv_prompts, k_folds, cfg.seed)

    if evaluators is None:
        evaluators = load_all_evaluators(cfg)

    fold_results: List[Dict[str, Any]] = []
    for i, (train_prompts_fold, val_prompts_fold) in enumerate(splits):
        print(f"\n=== Fold {i+1}/{k_folds}: entrenamiento con {len(train_prompts_fold)} prompts y validación con {len(val_prompts_fold)} prompts ===")
        fold_result = rl_training_loop_for_prompts(
            cfg,
            train_prompts_fold,
            val_prompts_fold,
            i,
            results_dir=results_dir,
            evaluators=evaluators,
        )
        fold_results.append(fold_result)

    best_fold = max(fold_results, key=lambda fr: fr["metrics"]["weighted"])
    print(f"\n🏆 Mejor fold: {best_fold['fold']+1} con puntuación ponderada {best_fold['metrics']['weighted']:.4f}")
    if best_fold.get('early_stopped', False):
        print(f"   ⚡ Este fold usó early stopping (se detuvo anticipadamente)")

    pipe_final = load_pipeline_with_lora(cfg, best_fold["weights_path"])

    clip_model = evaluators.clip_model
    weight = evaluators.weight
    bias = evaluators.bias
    tokenizer = evaluators.tokenizer
    pick_model = evaluators.pick_model
    hps_model = evaluators.hps_model

    best_train_prompts, best_val_prompts = splits[best_fold["fold"]]

    metrics_train = evaluate_prompts(
        pipe_final,
        best_train_prompts,
        cfg,
        clip_model,
        tokenizer,
        weight,
        bias,
        pick_model,
        hps_model,
    ) if best_train_prompts else {}
    metrics_val = evaluate_prompts(
        pipe_final,
        best_val_prompts,
        cfg,
        clip_model,
        tokenizer,
        weight,
        bias,
        pick_model,
        hps_model,
    )
    metrics_test = None
    if test_prompts:
        metrics_test = evaluate_prompts(
            pipe_final,
            test_prompts,
            cfg,
            clip_model,
            tokenizer,
            weight,
            bias,
            pick_model,
            hps_model,
        )

    summary_csv = os.path.join(results_dir, "cv_summary.csv")
    os.makedirs(results_dir, exist_ok=True)
    fieldnames = [
        "fold", "weighted", "aesthetic", "clip_align", "pickscore", "hpsv2",
        "early_stopped", "best_weighted"
    ]
    with open(summary_csv, "w", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for fr in fold_results:
            row = {
                "fold": fr["fold"] + 1,
                "weighted": fr["metrics"]["weighted"],
                "aesthetic": fr["metrics"]["aesthetic"],
                "clip_align": fr["metrics"]["clip_align"],
                "pickscore": fr["metrics"]["pickscore"],
                "hpsv2": fr["metrics"]["hpsv2"],
                "early_stopped": fr.get("early_stopped", False),
                "best_weighted": fr.get("best_weighted_score", 0.0),
            }
            writer.writerow(row)
        if metrics_train:
            writer.writerow({
                "fold": "final_train",
                "weighted": metrics_train.get("weighted", 0.0),
                "aesthetic": metrics_train.get("aesthetic", 0.0),
                "clip_align": metrics_train.get("clip_align", 0.0),
                "pickscore": metrics_train.get("pickscore", 0.0),
                "hpsv2": metrics_train.get("hpsv2", 0.0),
                "early_stopped": "",
                "best_weighted": "",
            })
        writer.writerow({
            "fold": "final_val",
            "weighted": metrics_val.get("weighted", 0.0),
            "aesthetic": metrics_val.get("aesthetic", 0.0),
            "clip_align": metrics_val.get("clip_align", 0.0),
            "pickscore": metrics_val.get("pickscore", 0.0),
            "hpsv2": metrics_val.get("hpsv2", 0.0),
            "early_stopped": "",
            "best_weighted": "",
        })
        if metrics_test is not None:
            writer.writerow({
                "fold": "final_test",
                "weighted": metrics_test.get("weighted", 0.0),
                "aesthetic": metrics_test.get("aesthetic", 0.0),
                "clip_align": metrics_test.get("clip_align", 0.0),
                "pickscore": metrics_test.get("pickscore", 0.0),
                "hpsv2": metrics_test.get("hpsv2", 0.0),
                "early_stopped": "",
                "best_weighted": "",
            })
    print(f"📊 Resumen de validación cruzada guardado en {summary_csv}")
    print("\n📈 Métricas finales del modelo seleccionado:")
    if metrics_train:
        print(f"  Entrenamiento: {metrics_train}")
    print(f"  Validación:   {metrics_val}")
    if metrics_test is not None:
        print(f"  Test:         {metrics_test}")

    try:
        plot_training_validation_metrics(
            best_fold.get("train_metrics_history", []),
            best_fold.get("val_metrics_history", []),
            results_dir,
            prefix=f"fold_{best_fold['fold']+1}"
        )
    except Exception as e:
        print(f"[Aviso] No se pudieron generar las gráficas de entrenamiento/validación: {e}")
    if metrics_test is not None:
        try:
            plot_test_metrics(
                metrics_test,
                results_dir,
                prefix=f"fold_{best_fold['fold']+1}"
            )
        except Exception as e:
            print(f"[Aviso] No se pudieron generar las gráficas de test: {e}")

    if return_results:
        return {
            "best_fold_index": best_fold["fold"],
            "best_fold_metrics": best_fold["metrics"],
            "best_weights_path": best_fold["weights_path"],
            "metrics_train": metrics_train,
            "metrics_val": metrics_val,
            "metrics_test": metrics_test,
        }
    return {}


def sample_hyperparameters(cfg: RLConfig) -> RLConfig:
    new_cfg = deepcopy(cfg)
    new_cfg.lora_rank = random.choice(cfg.lora_rank_options)
    new_cfg.lora_alpha = random.choice(cfg.lora_alpha_options)
    new_cfg.lora_dropout = random.choice(cfg.lora_dropout_options)
    new_cfg.lr = random.choice(cfg.lr_options)
    def pick_weight(options: Tuple[float, ...]) -> float:
        return random.choice(options) if options else random.random()
    w_a = pick_weight(cfg.w_aesthetic_options)
    w_c = pick_weight(cfg.w_clip_align_options)
    w_p = pick_weight(cfg.w_pickscore_options)
    w_h = pick_weight(cfg.w_hpsv2_options)
    total_w = w_a + w_c + w_p + w_h
    if total_w == 0:
        w_a = w_c = w_p = w_h = 0.25
        total_w = 1.0
    new_cfg.w_aesthetic = w_a / total_w
    new_cfg.w_clip_align = w_c / total_w
    new_cfg.w_pickscore = w_p / total_w
    new_cfg.w_hpsv2 = w_h / total_w
    return new_cfg


def random_search_cross_validation(cfg: RLConfig) -> Dict[str, Any]:
    assert cfg.param_search_n > 0, "La búsqueda aleatoria requiere param_search_n > 0"
    best_result: Optional[Dict[str, Any]] = None
    best_score: float = -float('inf')
    for i in range(cfg.param_search_n):
        print(f"\n⚙️  Búsqueda aleatoria {i+1}/{cfg.param_search_n}")
        sampled_cfg = sample_hyperparameters(cfg)
        print(
            f"   -> LoRA: rank={sampled_cfg.lora_rank}, alpha={sampled_cfg.lora_alpha}, dropout={sampled_cfg.lora_dropout}\n"
            f"   -> LR={sampled_cfg.lr:.1e}, Pesos recompensa=[{sampled_cfg.w_aesthetic:.2f}, {sampled_cfg.w_clip_align:.2f}, {sampled_cfg.w_pickscore:.2f}, {sampled_cfg.w_hpsv2:.2f}]"
        )
        try:
            result = cross_validation_training(sampled_cfg, return_results=True)
        except Exception as e:
            print(f"[Aviso] La combinación {i+1} falló: {e}")
            continue
        score = result.get("metrics_val", {}).get("weighted", 0.0) if result else 0.0
        if score > best_score:
            best_score = score
            best_result = {**result, "config": deepcopy(sampled_cfg)}
            print(f"✅ Nueva mejor combinación con score {score:.4f}")
        else:
            print(f"❌ Combinación descartada: score {score:.4f} (mejor hasta ahora {best_score:.4f})")
    print("\n🔍 Búsqueda aleatoria completada.")
    if best_result:
        cfg_best = best_result["config"]
        print(
            "\n🏁 Mejor configuración encontrada:\n"
            f"   -> LoRA rank={cfg_best.lora_rank}, alpha={cfg_best.lora_alpha}, dropout={cfg_best.lora_dropout}\n"
            f"   -> LR={cfg_best.lr:.1e}, Pesos recompensa=[{cfg_best.w_aesthetic:.2f}, {cfg_best.w_clip_align:.2f}, {cfg_best.w_pickscore:.2f}, {cfg_best.w_hpsv2:.2f}]\n"
            f"   -> Métrica de validación combinada: {best_score:.4f}"
        )
    else:
        print("❌ No se encontró ninguna configuración válida durante la búsqueda aleatoria.")
    return best_result or {}


def load_pipeline_with_lora(cfg: RLConfig, weights_path: str) -> DiffusionPipeline:
    pipe = load_unclip_pipeline(cfg)
    add_lora_to_decoder(pipe, cfg=cfg)
    if weights_path and os.path.exists(weights_path):
        try:
            payload = torch.load(weights_path, map_location=cfg.device)
            if isinstance(payload, dict) and "lora_state_dict" in payload:
                state_dict = payload["lora_state_dict"]
                print(
                    f"ℹ️ Cargando LoRA con rank={payload.get('lora_rank')}, "
                    f"alpha={payload.get('lora_alpha')}, dropout={payload.get('lora_dropout')}"
                )
            else:
                state_dict = payload

            get_decoder_unet(pipe).load_state_dict(state_dict, strict=False)
            print(f"✅ Pesos LoRA cargados desde {weights_path}")
        except Exception as e:
            print(f"❌ Error al cargar pesos LoRA desde {weights_path}: {e}")
    else:
        print(f"[Aviso] No se encontró el archivo de pesos {weights_path}; se utilizará el modelo base")
    return pipe


def plot_training_validation_metrics(
    train_history: List[Dict[str, float]],
    val_history: List[Dict[str, float]],
    output_dir: str,
    prefix: str,
) -> None:
    if not train_history or not val_history:
        return
    os.makedirs(output_dir, exist_ok=True)
    steps = list(range(1, len(train_history) + 1))
    metrics = ["aesthetic", "clip_align", "pickscore", "hpsv2", "weighted"]
    for metric in metrics:
        train_vals = [h.get(metric, 0.0) for h in train_history]
        val_vals = [h.get(metric, 0.0) for h in val_history]
        plt.figure()
        plt.plot(steps, train_vals, label=f"Train {metric}")
        plt.plot(steps, val_vals, label=f"Val {metric}")
        plt.xlabel("Paso")
        plt.ylabel(metric)
        plt.title(f"{prefix} {metric} vs Paso")
        plt.legend()
        filename = f"{prefix}_{metric}_curve.png"
        plt.savefig(os.path.join(output_dir, filename))
        plt.close()


def plot_test_metrics(metrics: Dict[str, float], output_dir: str, prefix: str) -> None:
    if not metrics:
        return
    os.makedirs(output_dir, exist_ok=True)
    metric_names = ["aesthetic", "clip_align", "pickscore", "hpsv2", "weighted"]
    values = [metrics.get(name, 0.0) for name in metric_names]
    plt.figure()
    plt.bar(metric_names, values)
    plt.xlabel("Métrica")
    plt.ylabel("Valor")
    plt.title(f"{prefix} Test Metrics")
    filename = f"{prefix}_test_metrics.png"
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()


def save_test_images(
    pipe: DiffusionPipeline,
    cfg: RLConfig,
    test_prompts: List[str],
    output_dir: str = "test_samples",
) -> None:
    os.makedirs(output_dir, exist_ok=True)
    batch_size = getattr(cfg, 'eval_batch_size', 2)
    print("\n" + "=" * 60)
    print("🎨 Guardando imágenes de test en: " + output_dir)
    print("📊 Total de prompts: " + str(len(test_prompts)))
    print("=" * 60 + "\n")
    for i in range(0, len(test_prompts), batch_size):
        batch_prompts = test_prompts[i:i + batch_size]
        with torch.no_grad():
            images = call_pipe_compat(
                pipe,
                batch_prompts,
                cfg,
                want_pt=True,
                override_decoder_steps=40,
                override_sr_steps=10,
            )
        for j, img in enumerate(images):
            prompt_idx = i + j
            if prompt_idx < len(test_prompts):
                prompt = test_prompts[prompt_idx]
                img_filename = f"test_img_{prompt_idx:04d}.png"
                img_save_path = os.path.join(output_dir, img_filename)
                save_image(img.cpu(), img_save_path)
                prompt_filename = f"test_img_{prompt_idx:04d}_prompt.txt"
                prompt_save_path = os.path.join(output_dir, prompt_filename)
                with open(prompt_save_path, 'w', encoding='utf-8') as f:
                    f.write(prompt)
                print(f"✅ [{prompt_idx+1}/{len(test_prompts)}] Imagen guardada: {img_save_path}")
                print(f"   📝 Prompt guardado: {prompt_save_path}")
    print("\n" + "=" * 60)
    print("✅ Todas las imágenes de test guardadas exitosamente!")
    print("   📁 Directorio: " + os.path.abspath(output_dir))
    print("=" * 60 + "\n")


def save_test_images_with_trained_lora(
    cfg: RLConfig,
    weights_path: str,
    test_prompts: List[str],
    output_dir: str = "test_samples",
) -> None:
    pipe = load_pipeline_with_lora(cfg, weights_path)
    save_test_images(pipe, cfg, test_prompts, output_dir)


def main():
    print("\n" + "=" * 80)
    print("⚙️  CONFIGURACIÓN DE LoRA - ENTRENAMIENTO RLHF KARLO")
    print("=" * 80)
    cfg = RLConfig()
    print("\n📝 Configurando parámetros de LoRA:")
    print(f"   ✅ lora_rank = {cfg.lora_rank}")
    print(f"   ✅ lora_alpha = {cfg.lora_alpha}")
    print(f"   ✅ lora_dropout = {cfg.lora_dropout}")
    cfg.device = "cuda:0" if torch.cuda.is_available() else "cpu"
    print(f"   ✅ device = {cfg.device}")
    cfg.train_decoder_steps = cfg.decoder_steps
    cfg.eval_decoder_steps = cfg.decoder_steps

    print("\n" + "=" * 80)
    print("CONFIGURACIÓN COMPLETA DE RUTAS")
    print("=" * 80)
    print(f"📁 Modelo: {cfg.model_path}")
    print(f"📁 Cache: {cfg.cache_dir}")
    print(f"📁 Prompts entrenamiento: {cfg.train_prompts_file}")
    print(f"📁 Prompts validación: {cfg.val_prompts_file}")
    print(f"📁 Evaluación: {cfg.eval_dir}")
    print(f"📁 PickScore: {cfg.pickscore_model_path}")
    print(f"📁 HPS v2: {cfg.hpsv2_model_path}")
    print("=" * 80)

    if cfg.param_search_n and cfg.param_search_n > 0:
        print("\n🔁 Ejecutando búsqueda aleatoria de hiperparámetros...")
        search_result = random_search_cross_validation(cfg)
        if search_result and 'best_weights_path' in search_result:
            test_prompts, _ = prepare_cv_datasets(cfg)
            if test_prompts:
                print("\n🎨 Generando imágenes de test con la mejor configuración encontrada...")
                save_test_images_with_trained_lora(cfg, search_result['best_weights_path'], test_prompts)
    else:
        print("\n🚀 INICIANDO VALIDACIÓN CRUZADA POR DEFECTO")
        cv_result = cross_validation_training(cfg, return_results=True)
        test_prompts, _ = prepare_cv_datasets(cfg)
        if cv_result and test_prompts:
            print("\n🎨 Generando imágenes de test con el mejor fold...")
            save_test_images_with_trained_lora(cfg, cv_result['best_weights_path'], test_prompts)


if __name__ == "__main__":
    main()

100%|██████████| 10/10 [00:08<00:00,  1.17it/s]


[UnCLIP] steps -> prior=25, decoder=25 (truncate_last_k=25), super_res=10


100%|██████████| 10/10 [00:08<00:00,  1.18it/s]


[UnCLIP] steps -> prior=25, decoder=25 (truncate_last_k=25), super_res=10


100%|██████████| 10/10 [00:03<00:00,  2.77it/s]


...

100%|██████████| 10/10 [00:03<00:00,  2.76it/s]


[UnCLIP] steps -> prior=25, decoder=25 (truncate_last_k=25), super_res=10


100%|██████████| 10/10 [00:03<00:00,  2.76it/s]


[UnCLIP] steps -> prior=25, decoder=25 (truncate_last_k=25), super_res=10


100%|██████████| 10/10 [00:03<00:00,  2.77it/s]
